In [7]:
import pandas as pd
import pyterrier as pt
import numpy as np
import os
from repro_eval.Evaluator import RpdEvaluator
from repro_eval.util import arp, arp_scores
import pytrec_eval
import yaml
if not pt.started():
    pt.init()

# Time Fuse
Fuse the new run with one old run by boosting old (known) documents up and new documents down.

In [39]:
def time_fuse(run_recent, run_old, _lambda=0.5):
    qid_ranking_groups = run_old.groupby('qid')
    qid_ranking_dict = {qid: list(ranking['docno']) for qid, ranking in qid_ranking_groups}
    
    def weigh(row):
        if not qid_ranking_dict.get(row['qid']):
            print("Could not find", row['qid'])
            
        if row['docno'] in qid_ranking_dict.get(row['qid'], []):
            return row['score'] * _lambda ** 2
        else:
            return row['score'] * (1-_lambda) ** 2           
    reranking = run_recent.copy()
    
    # min max normalization per topic
    reranking['score'] = reranking.groupby('qid')['score'].transform(lambda x : x / x.max())
    
    # weight if in old ranking
    reranking['score'] = reranking.progress_apply(weigh, axis=1)
    reranking = reranking.sort_values(['qid','score'], ascending=False).groupby('qid').head(1000)
    reranking['rank'] = reranking.groupby('qid')['score'].rank(ascending=False).astype(int)
    return reranking

# Lambda sweep

In [40]:
base_path = "data"
runs_path = "results/trec"
reranked_path = "results/fuse_time"

run_new_path = "CIR_BM25_D-t3_T-t3"
run_old_path = "CIR_BM25_D-t2_T-t3_extended"

with open("data/LongEval/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [52]:
run_new = pt.io.read_results(os.path.join(base_path, runs_path, run_new_path))
run_old = pt.io.read_results(os.path.join(base_path, runs_path, run_old_path))

with open(os.path.join(base_path, config["subcollections"]["t3"]["qrels"]["test"]), "r") as f_qrels:
    qrels = pytrec_eval.parse_qrel(f_qrels)
evaluator = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

results = {}
# for l in np.logspace(-3, -6, num=10):  # sweep from 1e-3 to 1e-12
for l in range(1, 11):
    _lambda =  l/10
    # _lambda = 0.5 + l
    run_reranked = time_fuse(run_new, run_old, _lambda=_lambda)
    results[_lambda] = {}
    
    # write results
    run_name = f'CIR_BM25_D-t3_T-t3_rr-t2-{l}'
    run_reranked_path = os.path.join(base_path, reranked_path, run_name)
    pt.io.write_results(run_reranked, run_reranked_path, format='trec', run_name=run_name)

    # evaluate
    rpd_eval = RpdEvaluator(run_b_orig_path=os.path.join(base_path, runs_path, run_new_path), run_b_rep_path=run_reranked_path)
    
    correlations = rpd_eval.ktau_union().get('baseline')
    correlation_scores = [x for x in list(correlations.values()) if ~np.isnan(x)]
    avg_tau = sum(correlation_scores) / len(correlation_scores)
    results[_lambda]["tau"] = avg_tau
    
    with open(run_reranked_path) as run_reranked:
        run = pytrec_eval.parse_run(run_reranked)
        scores = evaluator.evaluate(run)
        results[_lambda]["arp"] = arp_scores(scores)


 42%|████▏     | 247303/585414 [00:02<00:02, 123534.79it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


 43%|████▎     | 251143/585414 [00:02<00:02, 124120.52it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


 43%|████▎     | 249833/585414 [00:02<00:02, 119381.31it/s]

...

 43%|████▎     | 249526/585414 [00:02<00:02, 124278.15it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


 43%|████▎     | 253079/585414 [00:02<00:02, 126240.82it/s]

Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713
Could not find q012351539607713


100%|██████████| 585414/585414 [00:04<00:00, 117219.08it/s]


In [53]:
for l in results.keys():
    print("|", " | ".join([
        str(l), 
        str(round(results[l]["tau"], 4)), 
        str(round(results[l]["arp"]["P_10"], 4)), 
        str(round(results[l]["arp"]["bpref"], 4)), 
        str(round(results[l]["arp"]["ndcg"], 4))
        ]), "|")

| 0.1 | 0.0061 | 0.1214 | 0.4335 | 0.3139 |
| 0.2 | 0.0061 | 0.1214 | 0.4335 | 0.3139 |
| 0.3 | 0.0061 | 0.1214 | 0.4335 | 0.3139 |
| 0.4 | 0.0059 | 0.1217 | 0.4333 | 0.3153 |
| 0.5 | 1.0 | 0.1624 | 0.4373 | 0.3638 |
| 0.6 | 0.0134 | 0.1157 | 0.4196 | 0.2841 |
| 0.7 | 0.012 | 0.1125 | 0.4194 | 0.2764 |
| 0.8 | 0.0122 | 0.1125 | 0.4194 | 0.2763 |
| 0.9 | 0.0122 | 0.1125 | 0.4194 | 0.2763 |
| 1.0 | 0.0061 | 0.112 | 0.399 | 0.2657 |


# Filter Fuse

In [25]:
import numpy as np
from repro_eval.Evaluator import RpdEvaluator
from ranx import Run, fuse

In [26]:
def filter_and_fuse(run_recent, old_runs: list):
    qid_ranking_groups = run_recent.groupby('qid')
    qid_ranking_dict_recent = {qid: pd.Series(ranking['score'].values, ranking['docno']).to_dict() for qid, ranking in qid_ranking_groups}
    
    runs = [Run.from_dict(qid_ranking_dict_recent)]
    
    for run_old in old_runs:
        qid_ranking_groups = run_old.groupby('qid')
        qid_ranking_dict_old = {qid: pd.Series(ranking['score'].values, ranking['docno']).to_dict() for qid, ranking in qid_ranking_groups}
        for qid, ranking in qid_ranking_dict_old.items():
            docs_recent = qid_ranking_dict_recent.get(qid).keys()
            qid_ranking_dict_old[qid] = {docid: score for docid, score in ranking.items() if docid in docs_recent}
        runs.append(Run.from_dict(qid_ranking_dict_old))
    
    combined_run = fuse(runs = runs, method = "rrf")

    return combined_run

In [27]:
base_path = "data"
runs_path = "results/trec"
reranked_path = "results/filter_fuse"

run_new_path = "CIR_BM25_D-t3_T-t3"
run_old_path = "CIR_BM25_D-t2_T-t3_extended"

with open("data/LongEval/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [28]:
run_new = pt.io.read_results(os.path.join(base_path, runs_path, run_new_path))

old_runs = []
for name in os.listdir(os.path.join(base_path, runs_path)):
    if "T-t3" in name and name.endswith("extended"):
        run = pt.io.read_results(os.path.join(base_path, runs_path, name))
        old_runs.append(run)

In [29]:
# find core qids
topic_sets = []
for i in old_runs:
    topic_sets.append(set(i["qid"]))
topic_sets.append(set(run_new["qid"]))

core = set.intersection(*topic_sets)
print(len(core))

597


In [30]:
old_runs_cleaned = []
for run in old_runs:
    old_runs_cleaned.append(run[run["qid"].isin(core)])
    

In [31]:
old_runs = old_runs_cleaned

In [32]:
run_new_cleaned = run_new[run_new["qid"].isin(core)]

In [33]:
run_reranked = filter_and_fuse(run_new_cleaned, old_runs)

In [34]:
run_name = f'CIR_BM25_D-t3_T-t3_rr-ff'
run_reranked_path = os.path.join(base_path, reranked_path, run_name)
run_reranked.save(run_reranked_path,  kind='trec')

In [38]:
rpd_eval = RpdEvaluator(run_b_orig_path=os.path.join(base_path, runs_path, run_new_path), run_b_rep_path=run_reranked_path)

correlations = rpd_eval.ktau_union().get('baseline')
correlation_scores = [x for x in list(correlations.values()) if ~np.isnan(x)]

print("Avg. Kendall's tau: ", sum(correlation_scores) / len(correlation_scores))

with open(run_reranked_path) as run_reranked:
    run = pytrec_eval.parse_run(run_reranked)
    scores = evaluator.evaluate(run)
    print(round(arp_scores(scores)["bpref"], 4))
    print(round(arp_scores(scores)["P_10"], 4))
    print(round(arp_scores(scores)["ndcg"], 4))

Avg. Kendall's tau:  0.004242197558995947
0.4218
0.1062
0.2914
